In [1]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain_chroma import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import PromptTemplate
from langchain_community.vectorstores import FAISS
from dotenv import load_dotenv

import os

In [2]:
load_dotenv()
OPEN_API_KEY = os.getenv("OPENAI_API_KEY")

In [3]:
# Load Models
embedding = OpenAIEmbeddings(model="text-embedding-3-small")
llm = ChatOpenAI(model_name="gpt-4o-mini", max_tokens=200)

In [4]:
# Load pdf
pdf_directory = "os-sertoes.pdf"

loader = PyPDFLoader(pdf_directory, extract_images=False)
pages = loader.load_and_split()

In [5]:
# Quantity pages
len(pages)

658

In [14]:
# Splitter - create chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
    length_function=len,
    add_start_index=True,
    separators=["\n\n", "\n", ".", "?", "!"]
)

chunks = text_splitter.split_documents(pages)

In [15]:
# Save in vector db
db = Chroma.from_documents(chunks, embedding=embedding, persist_directory="text_index_simples_rag")

In [16]:
# Load DB
vectorDb = Chroma(persist_directory='text_index_simples_rag', embedding_function=embedding)

# Load retriever
retriever = vectorDb.as_retriever(search_kwars={"k": 3})

# Prompt
prompt = PromptTemplate.from_template(
    "Utilize o seguinte contexto extraído de documentos PDF para responder à pergunta:\n\n"
    "{context}\n\n"
    "Pergunta: {question}\n\n"
    "Resposta:"
)

chain = create_stuff_documents_chain(llm, prompt)

In [17]:
def ask(question):
    context = retriever.invoke(question)
    
     # Load question and document 
    result = chain.invoke({
        "context": context,
        "question": question
    })
    
    # Extract answer
    answer = result
    
    return answer, context

In [ ]:
user_question_1 = "Qual é a visão de Euclides da Cunha sobre o ambiente natural do sertão nordestino e como ele influencia a vida dos habitantes?"
answer_1, context_1 = ask(user_question_1)

print("Answer 1: ", answer_1)

Answer:  A visão de Euclides da Cunha sobre o ambiente natural do sertão nordestino é marcada por uma profunda ambivalência, refletida na descrição de um espaço que, apesar de suas riquezas e potencialidades, é também palco de severas adversidades. Ele reconhece as antíteses da natureza, apresentando o sertão como um vale fértil em algumas partes, caracterizado por um vasto pomar sem dono, mas que rapidamente se transforma em um cenário desolador e árido, onde a estiagem e a falta de chuvas criam um clima asfixiante e um solo empedrado.

Essa dualidade do ambiente natural influencia diretamente a vida dos habitantes do sertão. A escassez de chuvas e a ocorrência periódica de secas devastadoras moldam não apenas a economia local, que é fortemente centrada na agricultura, mas também a cultura e a forma de vida das pessoas. Os habitantes se veem em um constante


In [19]:
user_question_2 = "Quais são as principais características da população sertaneja descritas por Euclides da Cunha? Como ele relaciona essas características com o ambiente em que vivem?"
answer_2, context_2 = ask(user_question_2)

print("Answer 2: ", answer_2)
print("Context 2: ", context_2)

Answer 2:  As principais características da população sertaneja descritas por Euclides da Cunha incluem a variedade e complexidade étnica, marcada pela mistura de brancos, negros e mulatos. Essa diversidade contribui para uma riqueza de matizes, tanto na aparência física quanto nos aspectos do caráter dos indivíduos. Ele ressalta que essa população possui tendências coletivas que se manifestam em crenças ingênuas e místicas, abrangendo desde o fetichismo até distortas interpretações de práticas católicas.

Euclides da Cunha liga essas características da população ao ambiente severo e desigual em que vivem. O sertão é retratado como um espaço onde as condições ambientais adversas, como a escassez de recursos e as rigorosas condições climáticas, contribuem para a formação do caráter e das crenças dos sertanejos. O autor argumenta que a vida sertaneja se caracteriza pela indisciplina e pela receptividade a influências externas, levando a um
Context 2:  [Document(id='21bd9df7-79a9-4d03-954

In [20]:
user_question_3 = "Qual foi o contexto histórico e político que levou à Guerra de Canudos, segundo Euclides da Cunha?"
answer_3, context_3 = ask(user_question_3)

print("Answer 3: ", answer_3)
print("Context 3: ", context_3)

Answer 3:  A Guerra de Canudos, segundo Euclides da Cunha, ocorreu em um contexto histórico e político marcado pela transição do Brasil de uma monarquia para uma república e pelas tensões sociais e econômicas que essa mudança gerou. O conflito se desenrolou entre 1896 e 1897 no interior da Bahia, em um cenário de agitação social e rivalidades políticas. Na época, o Brasil estava sob a liderança do governo republicano de Floriano Peixoto, que enfrentava resistência de grupos conservadores e monarquistas.

A figura central da guerra foi o beato Antônio Conselheiro, que atraiu os habitantes de Canudos, formando uma comunidade que buscava apoio em valores religiosos e um sistema de liderança semelhante a um império sob um chefe messiânico. Essa resistência expressava um descontentamento profundo com a nova ordem republicana, que era vista como uma abstração distante e inacessível, principalmente para as camadas mais pobres da
Context 3:  [Document(id='fdd7eb24-2a99-4196-b243-8f0b36cc55b1',

In [21]:
user_question_4 = "Como Euclides da Cunha descreve a figura de Antônio Conselheiro e seu papel na Guerra de Canudos?"
answer_4, context_4 = ask(user_question_4)

print("Answer 4: ", answer_4)
print("Context 4: ", context_4)

Answer 4:  Euclides da Cunha descreve Antônio Conselheiro como uma figura carismática e enigmática, com características de um apóstolo desnorteado. Ele é retratado como um líder que atrai centenas de seguidores que preferem ouvir suas ordens às do vigário local. Conselheiro é visto como um agente de transformação no sertão, promovendo a construção de uma capela que, embora considerada um "melhoramento" para a comunidade, é acompanhada por sacrifícios excessivos. Essa situação gera temores de grandes desgraças, mostrando a complexidade das emoções e tensões sociais presentes na região.

Além disso, sua figura é envolta em uma aura de misticismo e lenda, influenciando profundamente aqueles que o cercam. Ele é creditado com poderes quase sobrenaturais, e sua imagem se torna uma espécie de ícone entre os combatentes da Guerra de Canudos. A narrativa de Euclides da Cunha destaca
Context 4:  [Document(id='e0b0f300-ab8c-4f11-89c9-6131129cd941', metadata={'creationdate': '2014-05-09T00:33:41-0

In [22]:
user_question_5 = "Quais são os principais aspectos da crítica social e política presentes em 'Os Sertões'? Como esses aspectos refletem a visão do autor sobre o Brasil da época?"
answer_5, context_5 = ask(user_question_5)

print("Answer 5: ", answer_5)
print("Context 5: ", context_5)

Answer 5:  Os principais aspectos da crítica social e política presentes em "Os Sertões" de Euclides da Cunha estão relacionados à desigualdade social, à questão da identidade nacional e às tensões entre diferentes grupos raciais e sociais, especialmente no contexto do sertão nordestino. A obra evidencia a luta do sertanejo, apresentado como o "deserdado da fortuna", em um cenário de opressão e marginalização, refletindo a profunda desigualdade existente na sociedade brasileira da época.

A narrativa de Euclides da Cunha vai além de uma simples descrição dos eventos da Campanha de Canudos; ela se transforma em uma crítica das condições socioeconômicas que levaram ao conflito. A obra expõe as fragilidades da civilização urbana frente à dureza da vida no sertão e demonstra uma preocupação com a identidade nacional, ao mostrar como as sub-raças sertanejas estão em risco pela modernização e pelas correntes migratórias que ameaçam sua existência. O
Context 5:  [Document(id='9aedad6f-8127-4a